# Delta Sharing Python Integration

This notebook demonstrates how to connect to and use data from a Delta Share using the native Python `delta-sharing` library. This approach provides full functionality and handles advanced Delta features like Deletion Vectors.

## Features
- ✅ Native Python implementation (no R/Python integration complexity)
- ✅ Full Delta Sharing functionality 
- ✅ Supports Deletion Vectors and advanced Delta features
- ✅ Direct data loading without manual file handling
- ✅ Built-in authentication and token management

## 1. Install and Import Delta Sharing Libraries

First, we'll install the required packages and import the necessary libraries.

In [1]:
# Install delta-sharing if not already installed
import subprocess
import sys

try:
    import delta_sharing
    print("✅ delta-sharing already installed")
except ImportError:
    print("Installing delta-sharing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "delta-sharing>=1.0.0"])
    import delta_sharing
    print("✅ delta-sharing installed successfully")

# Import required libraries
import pandas as pd
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("📦 All libraries imported successfully")
print(f"delta-sharing version: {delta_sharing.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"Python version: {sys.version}")

✅ delta-sharing already installed
📦 All libraries imported successfully
delta-sharing version: 1.3.3
pandas version: 2.3.3
Python version: 3.11.13 (main, Jun 12 2025, 12:41:02) [Clang 20.1.4 ]


## 2. Configure Delta Sharing Client

Load the configuration file and set up the client connection parameters.

In [ ]:
# Configuration file path
config_path = "../config.share"

# Check if config file exists
if not os.path.exists(config_path):
    # Try alternative paths
    alternative_paths = [
        "config.share",
        "/home/rstudio/workspace/config.share",
        "notebooks/config.share"
    ]
    
    for path in alternative_paths:
        if os.path.exists(path):
            config_path = path
            break
    else:
        raise FileNotFoundError(f"Config file not found. Tried: {[config_path] + alternative_paths}")

print(f"✅ Using config file: {config_path}")
print(f"📁 File exists: {os.path.exists(config_path)}")

# Load and display configuration (without sensitive data)
with open(config_path, 'r') as f:
    config = json.load(f)

print(f"\n📊 Configuration details:")
print(f"Endpoint: {config['endpoint']}")
print(f"Credentials version: {config['shareCredentialsVersion']}")
print(f"Token expires: {config['expirationTime']}")

# Check token expiration
try:
    expiry_time = datetime.fromisoformat(config['expirationTime'].replace('Z', '+00:00'))
    time_until_expiry = expiry_time - datetime.now().replace(tzinfo=expiry_time.tzinfo)
    hours_remaining = time_until_expiry.total_seconds() / 3600
    
    print(f"⏰ Hours until expiry: {hours_remaining:.1f}")
    
    if hours_remaining < 24:
        print("⚠️  WARNING: Token expires within 24 hours!")
    elif hours_remaining < 1:
        print("🔴 CRITICAL: Token expires within 1 hour!")
except Exception as e:
    print(f"⚠️  Could not parse expiry time: {e}")

print(f"\n✅ Configuration loaded successfully")

✅ Using config file: ../config.share
📁 File exists: True

📊 Configuration details:
Endpoint: https://norwayeast.azuredatabricks.net/api/2.0/delta-sharing/metastores/a0334db1-3d6f-43da-9137-ce2562cc9873
Credentials version: 1
Token expires: 2025-11-12T11:46:45.076Z
⏰ Hours until expiry: 21.1
⚠️  WARNING: Token expires within 24 hours!

✅ Configuration loaded successfully


## 3. Connect to Delta Sharing Server

Create the Delta Sharing client and establish connection.

In [3]:
# Create Delta Sharing client
try:
    client = delta_sharing.SharingClient(config_path)
    print("✅ Delta Sharing client created successfully")
    print(f"Client type: {type(client)}")
    
    # Test connection by trying to list shares
    print("\n🔗 Testing connection...")
    shares = client.list_shares()
    print(f"✅ Connection successful - found {len(shares)} share(s)")
    
except Exception as e:
    print(f"❌ Error creating client: {e}")
    print("\n🛠️  Troubleshooting:")
    print("1. Check that config.share file is valid JSON")
    print("2. Verify network connectivity to the endpoint")
    print("3. Ensure token is not expired")
    raise

✅ Delta Sharing client created successfully
Client type: <class 'delta_sharing.delta_sharing.SharingClient'>

🔗 Testing connection...
✅ Connection successful - found 1 share(s)


## 4. List Available Shares and Schemas

Explore the available shares, schemas, and tables in the Delta Sharing server.

In [7]:
# List all available shares
print("🔍 Listing available shares:")
shares = client.list_shares()

for i, share in enumerate(shares, 1):
    print(f"{i}. {share.name}")

# Select first share for exploration
if shares:
    selected_share_obj = shares[0]  # Keep the Share object
    selected_share = selected_share_obj.name  # Extract name for display
    print(f"\n✅ Using share: {selected_share}")
    
    # List schemas in the selected share (pass the Share object, not the name string)
    print(f"\n🗂️  Schemas in '{selected_share}':")
    schemas = client.list_schemas(selected_share_obj)  # Pass Share object
    
    for i, schema in enumerate(schemas, 1):
        print(f"{i}. {schema.name}")
    
    # Select first schema
    if schemas:
        selected_schema_obj = schemas[0]  # Keep the Schema object
        selected_schema = selected_schema_obj.name  # Extract name for display
        print(f"\n✅ Using schema: {selected_schema}")
        
        # List tables in the selected schema using list_tables method
        print(f"\n📊 Tables in '{selected_schema}':")
        tables = client.list_tables(selected_schema_obj)  # Pass Schema object
        
        for i, table in enumerate(tables, 1):
            table_name = table.name if hasattr(table, 'name') else str(table)
            share_name = table.share if hasattr(table, 'share') else selected_share
            schema_name = table.schema if hasattr(table, 'schema') else selected_schema
            print(f"{i}. {table_name} (Share: {share_name}, Schema: {schema_name})")
        
        # Store information for next steps
        if tables:
            selected_table = tables[0].name if hasattr(tables[0], 'name') else str(tables[0])
            print(f"\n✅ Using table: {selected_table}")
            
            # Store variables for later use
            share_name = selected_share
            schema_name = selected_schema  
            table_name = selected_table
            
            print(f"\n📝 Summary:")
            print(f"   Share: {share_name}")
            print(f"   Schema: {schema_name}")
            print(f"   Table: {table_name}")
        else:
            print("❌ No tables found in this schema")
    else:
        print("❌ No schemas found in this share")
else:
    print("❌ No shares available")

🔍 Listing available shares:
1. test_delta_share_natur_i_vern

✅ Using share: test_delta_share_natur_i_vern

🗂️  Schemas in 'test_delta_share_natur_i_vern':
1. gold_natur_i_verneomraader

✅ Using schema: gold_natur_i_verneomraader

📊 Tables in 'gold_natur_i_verneomraader':
1. d_variabler (Share: test_delta_share_natur_i_vern, Schema: gold_natur_i_verneomraader)

✅ Using table: d_variabler

📝 Summary:
   Share: test_delta_share_natur_i_vern
   Schema: gold_natur_i_verneomraader
   Table: d_variabler
1. d_variabler (Share: test_delta_share_natur_i_vern, Schema: gold_natur_i_verneomraader)

✅ Using table: d_variabler

📝 Summary:
   Share: test_delta_share_natur_i_vern
   Schema: gold_natur_i_verneomraader
   Table: d_variabler


## 5. Load Shared Tables as DataFrames

Load the Delta table data directly into pandas DataFrames. This approach handles Deletion Vectors automatically.

In [9]:
# Load data from the Delta table
if 'table_name' in locals():
    print(f"📥 Loading data from table: {table_name}")
    
    # Method 1: Using the config file with table reference
    try:
        # Format: config_file#share.schema.table
        table_url = f"{config_path}#{share_name}.{schema_name}.{table_name}"
        print(f"🔗 Table URL: {table_url}")
        
        print("⏳ Loading data (this may take a moment)...")
        df = delta_sharing.load_as_pandas(table_url)
        
        print(f"✅ Data loaded successfully!")
        print(f"📊 Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
        print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        
    except Exception as e:
        print(f"❌ Method 1 failed: {e}")
        
        # Method 2: Using client and table reference
        try:
            print("\n🔄 Trying alternative method with client...")
            
            # Create table reference
            table_ref = delta_sharing.Table(name=table_name, share=share_name, schema=schema_name)
            df = delta_sharing.load_as_pandas(table_ref, client)
            
            print(f"✅ Data loaded successfully with method 2!")
            print(f"📊 Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
            
        except Exception as e2:
            print(f"❌ Method 2 also failed: {e2}")
            print(f"\n🛠️  Troubleshooting tips:")
            print(f"1. Verify the table name: {table_name}")
            print(f"2. Check share access permissions")
            print(f"3. Ensure network connectivity")
            df = None
    
    # Display basic information about the loaded data
    if 'df' in locals() and df is not None:
        print(f"\n📋 Data Summary:")
        print(f"Column names: {list(df.columns)}")
        print(f"Data types:")
        for col, dtype in df.dtypes.items():
            print(f"  {col}: {dtype}")
        
        print(f"\n🔍 Last 5 rows:")
        display(df.tail())
        
        print(f"\n📈 Basic statistics:")
        display(df.describe())
    
else:
    print("❌ No table selected. Please run the previous cell to select a table.")

📥 Loading data from table: d_variabler
🔗 Table URL: ../config.share#test_delta_share_natur_i_vern.gold_natur_i_verneomraader.d_variabler
⏳ Loading data (this may take a moment)...
✅ Data loaded successfully!
📊 Dataset shape: 2,087,275 rows × 17 columns
✅ Data loaded successfully!
📊 Dataset shape: 2,087,275 rows × 17 columns
💾 Memory usage: 1598.5 MB

📋 Data Summary:
Column names: ['variabel_key', 'kode_variabeltype_key', 'nin_hovedtypegruppe', 'nin_hovedtype', 'gradient_kode', 'gradient_kode_beskrivelse', 'trinn_ulkm', 'trinn_definisjon', 'trinn_beskrivelse_ulkm', 'gruppe', 'gruppe_beskrivelse', 'tema', 'tema_beskrivelse', 'navn', 'navn_beskrivelse', 'trinn_variabler', 'trinn_beskrivelse_variabler']
Data types:
  variabel_key: object
  kode_variabeltype_key: object
  nin_hovedtypegruppe: object
  nin_hovedtype: float64
  gradient_kode: object
  gradient_kode_beskrivelse: object
  trinn_ulkm: float64
  trinn_definisjon: object
  trinn_beskrivelse_ulkm: object
  gruppe: object
  gruppe_b

,variabel_key,kode_variabeltype_key,nin_hovedtypegruppe,nin_hovedtype,gradient_kode,gradient_kode_beskrivelse,trinn_ulkm,trinn_definisjon,trinn_beskrivelse_ulkm,gruppe,gruppe_beskrivelse,tema,tema_beskrivelse,navn,navn_beskrivelse,trinn_variabler,trinn_beskrivelse_variabler
2087270,FFFE3360-A7A3-4E46-ABC2-4C8F94B6AB18,0,None,NaN,None,None,NaN,None,None,7,Tilstandsvariasjon,FA,Fremmedartsinnslag,None,None,0,Uten fremmedarter
2087271,FFFE7A10-9E22-42CB-B1E3-888F1F7EF639,0,None,NaN,None,None,NaN,None,None,7,Tilstandsvariasjon,FA,Fremmedartsinnslag,None,None,0,Uten fremmedarter
2087272,FFFF2FA0-7E3A-4CCF-947D-2992853CD39E,0,None,NaN,None,None,NaN,None,None,5,Menneskeskapte objekter,XG,Annen løs gjenstand,SM,Små objekter,1,0 – 1/16
2087273,FFFF60BD-BD5C-4088-BC1B-82B2E924FCA6,0,None,NaN,None,None,NaN,None,None,7,Tilstandsvariasjon,FA,Fremmedartsinnslag,None,None,0,Uten fremmedarter
2087274,FFFF7803-9D16-4711-860E-FCB3CFB35497,0,None,NaN,None,None,NaN,None,None,7,Tilstandsvariasjon,JB,Jord-bruk (aktuell bruk av jord),BA,Aktuell bruksintensitet,1,ikke i bruk



📈 Basic statistics:


,nin_hovedtype,trinn_ulkm
count,257546.000000,257546.000000
mean,9.242601,1.303689
std,11.145309,0.626470
min,1.000000,1.000000
25%,4.000000,1.000000
50%,4.000000,1.000000
75%,4.000000,1.000000
max,45.000000,6.000000


## 6. Query and Manipulate Shared Data

Perform data operations, filtering, and analysis on the loaded Delta table data.

In [ ]:
# Data exploration and manipulation
if 'df' in locals() and df is not None:
    print("🔬 Data Exploration and Analysis")
    print("=" * 50)
    
    # 1. Basic data quality checks
    print(f"\n1️⃣ Data Quality Overview:")
    print(f"   Total rows: {len(df):,}")
    print(f"   Total columns: {len(df.columns)}")
    print(f"   Missing values: {df.isnull().sum().sum():,}")
    print(f"   Duplicate rows: {df.duplicated().sum():,}")
    
    # 2. Missing values per column
    missing_data = df.isnull().sum()
    if missing_data.sum() > 0:
        print(f"\n🔍 Missing values by column:")
        missing_cols = missing_data[missing_data > 0].sort_values(ascending=False)
        for col, count in missing_cols.items():
            pct = (count / len(df)) * 100
            print(f"   {col}: {count:,} ({pct:.1f}%)")
    else:
        print(f"\n✅ No missing values found!")
    
    # 3. Data types and memory usage
    print(f"\n📊 Memory usage by column:")
    memory_usage = df.memory_usage(deep=True)
    for col, usage in memory_usage.items():
        print(f"   {col}: {usage / 1024**2:.2f} MB")
    
    # 4. Numeric columns analysis
    numeric_cols = df.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Numeric columns summary:")
        display(df[numeric_cols].describe())
        
        # Show correlations if multiple numeric columns
        if len(numeric_cols) > 1:
            print(f"\n🔗 Correlations between numeric columns:")
            corr_matrix = df[numeric_cols].corr()
            display(corr_matrix)
    
    # 5. Categorical columns analysis  
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) > 0:
        print(f"\n📝 Categorical columns analysis:")
        for col in categorical_cols[:5]:  # Show first 5 categorical columns
            unique_count = df[col].nunique()
            print(f"\n   Column: {col}")
            print(f"   Unique values: {unique_count}")
            if unique_count <= 10:
                value_counts = df[col].value_counts()
                print(f"   Value distribution:")
                for value, count in value_counts.head().items():
                    pct = (count / len(df)) * 100
                    print(f"     {value}: {count:,} ({pct:.1f}%)")
    
    # 6. Create some sample visualizations
    print(f"\n📊 Sample Data Visualizations:")
    
    # Import visualization libraries
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        plt.style.use('default')
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle(f'Data Analysis: {table_name}', fontsize=16)
        
        # Plot 1: Missing values heatmap (if any missing data)
        if df.isnull().sum().sum() > 0:
            sns.heatmap(df.isnull(), ax=axes[0,0], cbar=True, yticklabels=False)
            axes[0,0].set_title('Missing Values Pattern')
        else:
            axes[0,0].text(0.5, 0.5, 'No Missing Values', ha='center', va='center')
            axes[0,0].set_title('Missing Values Status')
        
        # Plot 2: Data types distribution
        dtype_counts = df.dtypes.value_counts()
        axes[0,1].pie(dtype_counts.values, labels=dtype_counts.index, autopct='%1.1f%%')
        axes[0,1].set_title('Data Types Distribution')
        
        # Plot 3: Numeric column histogram (first numeric column)
        if len(numeric_cols) > 0:
            col = numeric_cols[0]
            df[col].hist(bins=30, ax=axes[1,0])
            axes[1,0].set_title(f'Distribution: {col}')
            axes[1,0].set_xlabel(col)
            axes[1,0].set_ylabel('Frequency')
        else:
            axes[1,0].text(0.5, 0.5, 'No Numeric Columns', ha='center', va='center')
            axes[1,0].set_title('Numeric Data')
        
        # Plot 4: Categorical column bar chart (first categorical column)
        if len(categorical_cols) > 0:
            col = categorical_cols[0]
            top_values = df[col].value_counts().head(10)
            top_values.plot(kind='bar', ax=axes[1,1])
            axes[1,1].set_title(f'Top Values: {col}')
            axes[1,1].tick_params(axis='x', rotation=45)
        else:
            axes[1,1].text(0.5, 0.5, 'No Categorical Columns', ha='center', va='center')
            axes[1,1].set_title('Categorical Data')
        
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("📊 Matplotlib/Seaborn not available. Install with: pip install matplotlib seaborn")
    
    print(f"\n✅ Data exploration completed!")
    
else:
    print("❌ No data available for analysis. Please load data first.")

### Advanced Querying

Demonstrate advanced querying capabilities with filters and limits.

In [ ]:
# Advanced querying with delta-sharing
if 'table_url' in locals():
    print("🔍 Advanced Querying Examples")
    print("=" * 40)
    
    # Example 1: Load with limit
    print(f"\n1️⃣ Loading first 100 rows only:")
    try:
        df_limited = delta_sharing.load_as_pandas(table_url, limit=100)
        print(f"   ✅ Loaded {len(df_limited):,} rows")
        print(f"   Columns: {list(df_limited.columns)}")
    except Exception as e:
        print(f"   ❌ Limit query failed: {e}")
    
    # Example 2: Load with predicates (filtering)
    print(f"\n2️⃣ Loading with predicates (if supported):")
    
    # Note: Predicates depend on the specific table schema
    # Here are some common examples you can adapt:
    
    sample_predicates = [
        # "column_name > 100",
        # "status = 'ACTIVE'",
        # "date >= '2023-01-01'",
    ]
    
    if sample_predicates:
        for predicate in sample_predicates:
            try:
                print(f"   Trying predicate: {predicate}")
                df_filtered = delta_sharing.load_as_pandas(
                    table_url, 
                    predicates=[predicate]
                )
                print(f"   ✅ Filtered data: {len(df_filtered):,} rows")
                break
            except Exception as e:
                print(f"   ❌ Predicate '{predicate}' failed: {e}")
    else:
        print("   ℹ️  No predicates defined. Add column-specific filters above.")
    
    # Example 3: Load specific version (if versioning is supported)
    print(f"\n3️⃣ Loading specific table version:")
    try:
        # Most recent version (version=0 means latest)
        df_version = delta_sharing.load_as_pandas(table_url, version=0)
        print(f"   ✅ Latest version loaded: {len(df_version):,} rows")
    except Exception as e:
        print(f"   ❌ Version loading failed: {e}")
        print("   ℹ️  Version support may not be available for this table")
    
    # Example 4: Using pandas for post-processing
    if 'df' in locals() and df is not None:
        print(f"\n4️⃣ Post-processing with pandas:")
        
        # Basic filtering
        non_null_rows = len(df.dropna())
        print(f"   Rows without missing values: {non_null_rows:,}")
        
        # Column-specific operations
        numeric_cols = df.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            col = numeric_cols[0]
            print(f"   Statistics for '{col}':")
            print(f"     Mean: {df[col].mean():.2f}")
            print(f"     Median: {df[col].median():.2f}")
            print(f"     Min: {df[col].min():.2f}")
            print(f"     Max: {df[col].max():.2f}")
        
        # Grouping operations
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns
        if len(categorical_cols) > 0 and len(numeric_cols) > 0:
            cat_col = categorical_cols[0]
            num_col = numeric_cols[0]
            try:
                grouped = df.groupby(cat_col)[num_col].agg(['count', 'mean', 'std'])
                print(f"\n   Grouped statistics ({cat_col} vs {num_col}):")
                display(grouped.head())
            except Exception as e:
                print(f"   ❌ Grouping failed: {e}")
    
    print(f"\n✅ Advanced querying examples completed!")

else:
    print("❌ No table URL available. Please load a table first.")

## 7. Handle Authentication and Security

Best practices for authentication, token management, and secure connections.

In [ ]:
# Authentication and Security Best Practices
print("🔐 Authentication and Security Status")
print("=" * 45)

# 1. Check token expiration status
if 'config' in locals():
    print(f"\n1️⃣ Token Status:")
    try:
        expiry_time = datetime.fromisoformat(config['expirationTime'].replace('Z', '+00:00'))
        current_time = datetime.now().replace(tzinfo=expiry_time.tzinfo)
        time_remaining = expiry_time - current_time
        
        hours_remaining = time_remaining.total_seconds() / 3600
        days_remaining = time_remaining.days
        
        print(f"   Expires: {expiry_time.strftime('%Y-%m-%d %H:%M:%S %Z')}")
        print(f"   Time remaining: {days_remaining} days, {hours_remaining % 24:.1f} hours")
        
        # Security alerts
        if hours_remaining < 1:
            print("   🔴 CRITICAL: Token expires within 1 hour!")
        elif hours_remaining < 24:
            print("   🟡 WARNING: Token expires within 24 hours")
        elif days_remaining < 7:
            print("   🟠 NOTICE: Token expires within 1 week")
        else:
            print("   ✅ Token is valid for more than a week")
            
    except Exception as e:
        print(f"   ❌ Could not parse token expiration: {e}")

# 2. Connection security information
print(f"\n2️⃣ Connection Security:")
endpoint = config.get('endpoint', 'Unknown')
print(f"   Endpoint: {endpoint}")

if endpoint.startswith('https://'):
    print("   ✅ Secure HTTPS connection")
else:
    print("   ⚠️  Warning: Non-HTTPS endpoint detected")

# Bearer token security (don't expose the actual token)
bearer_token = config.get('bearerToken', '')
if bearer_token:
    print(f"   ✅ Bearer token present ({len(bearer_token)} characters)")
    # Check token format
    if bearer_token.startswith('dapi'):
        print("   ✅ Databricks API token format detected")
    else:
        print("   ℹ️  Custom token format")
else:
    print("   ❌ No bearer token found!")

# 3. Best practices checklist
print(f"\n3️⃣ Security Best Practices Checklist:")
practices = [
    ("Store config files securely", "✅" if os.path.exists(config_path) else "❌"),
    ("Use environment variables for secrets", "ℹ️"),
    ("Regularly rotate access tokens", "ℹ️"),
    ("Monitor token expiration", "✅" if hours_remaining > 24 else "⚠️"),
    ("Use HTTPS connections only", "✅" if endpoint.startswith('https://') else "❌"),
    ("Limit token scope/permissions", "ℹ️"),
    ("Audit data access logs", "ℹ️")
]

for practice, status in practices:
    print(f"   {status} {practice}")

# 4. Token refresh guidance
print(f"\n4️⃣ Token Refresh Guidance:")
print("   When your token approaches expiration:")
print("   1. Contact your data provider for a new token")
print("   2. Update the config.share file with new credentials")
print("   3. Test the connection with the new token")
print("   4. Consider automating token renewal if possible")

# 5. Environment setup for production
print(f"\n5️⃣ Production Environment Setup:")
print("   For production use, consider:")
print("   1. Use environment variables instead of config files:")
print("      export DELTA_SHARING_ENDPOINT='...'")
print("      export DELTA_SHARING_TOKEN='...'")
print("   2. Implement automatic token refresh")
print("   3. Set up monitoring for failed connections")
print("   4. Use secrets management systems (AWS Secrets Manager, etc.)")
print("   5. Implement retry logic with exponential backoff")

# 6. Security monitoring
print(f"\n6️⃣ Security Monitoring:")
if 'client' in locals():
    try:
        # Test connection health
        test_shares = client.list_shares()
        print(f"   ✅ Connection health check passed")
        print(f"   📊 Accessible shares: {len(test_shares)}")
    except Exception as e:
        print(f"   ❌ Connection health check failed: {e}")
        print(f"   🛠️  This may indicate token expiration or network issues")

print(f"\n✅ Security assessment completed!")

# Example of secure environment variable usage
print(f"\n💡 Example: Using environment variables")
print("""
import os
import json

# Create config from environment variables
def create_config_from_env():
    return {
        "shareCredentialsVersion": 1,
        "endpoint": os.environ.get("DELTA_SHARING_ENDPOINT"),
        "bearerToken": os.environ.get("DELTA_SHARING_TOKEN"),
        "expirationTime": os.environ.get("DELTA_SHARING_EXPIRY")
    }

# Usage:
# config = create_config_from_env()
# client = delta_sharing.SharingClient(config)
""")

## 8. Save and Export Data

Save the loaded data in various formats for further analysis and sharing.

In [10]:
# Save and export the loaded data
if 'df' in locals() and df is not None:
    print("💾 Saving and Exporting Data")
    print("=" * 35)
    
    # Create output directory
    output_dir = "data/delta-share-downloads"
    os.makedirs(output_dir, exist_ok=True)
    print(f"📁 Output directory: {output_dir}")
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_filename = f"{table_name}_{timestamp}"
    
    print(f"\n📊 Dataset info:")
    print(f"   Table: {table_name}")
    print(f"   Rows: {len(df):,}")
    print(f"   Columns: {len(df.columns)}")
    print(f"   Size: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # 1. Save as Parquet (recommended for large datasets)
    try:
        parquet_file = os.path.join(output_dir, f"{base_filename}.parquet")
        df.to_parquet(parquet_file, index=False)
        file_size = os.path.getsize(parquet_file) / 1024**2
        print(f"\n✅ Saved as Parquet: {parquet_file} ({file_size:.1f} MB)")
    except Exception as e:
        print(f"\n❌ Parquet save failed: {e}")
    
    # 2. Save as CSV (for compatibility)
    try:
        csv_file = os.path.join(output_dir, f"{base_filename}.csv")
        df.to_csv(csv_file, index=False)
        file_size = os.path.getsize(csv_file) / 1024**2
        print(f"✅ Saved as CSV: {csv_file} ({file_size:.1f} MB)")
    except Exception as e:
        print(f"❌ CSV save failed: {e}")
    
    # 3. Save as Excel (for small to medium datasets)
    if len(df) <= 50000:  # Excel row limit consideration
        try:
            excel_file = os.path.join(output_dir, f"{base_filename}.xlsx")
            df.to_excel(excel_file, index=False, engine='openpyxl')
            file_size = os.path.getsize(excel_file) / 1024**2
            print(f"✅ Saved as Excel: {excel_file} ({file_size:.1f} MB)")
        except Exception as e:
            print(f"❌ Excel save failed (install openpyxl if needed): {e}")
    else:
        print(f"⚠️  Skipping Excel export (dataset too large: {len(df):,} rows)")
    
    # 4. Save metadata as JSON
    try:
        metadata = {
            "table_name": table_name,
            "share_name": share_name,
            "schema_name": schema_name,
            "export_timestamp": datetime.now().isoformat(),
            "row_count": len(df),
            "column_count": len(df.columns),
            "columns": list(df.columns),
            "dtypes": {col: str(dtype) for col, dtype in df.dtypes.items()},
            "memory_usage_mb": df.memory_usage(deep=True).sum() / 1024**2,
            "has_missing_values": df.isnull().any().any(),
            "missing_value_count": int(df.isnull().sum().sum())
        }
        
        metadata_file = os.path.join(output_dir, f"{base_filename}_metadata.json")
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        print(f"✅ Saved metadata: {metadata_file}")
        
    except Exception as e:
        print(f"❌ Metadata save failed: {e}")
    
    # 5. Check for spatial data and save as GeoParquet if applicable
    potential_geom_cols = [col for col in df.columns 
                          if any(geo_word in col.lower() 
                                for geo_word in ['geom', 'geometry', 'shape', 'wkt', 'wkb'])]
    
    if potential_geom_cols:
        print(f"\n🗺️  Potential geometry columns detected: {potential_geom_cols}")
        try:
            import geopandas as gpd
            
            # Try to convert to GeoDataFrame
            for geom_col in potential_geom_cols:
                try:
                    # Attempt to create geometry from various formats
                    if df[geom_col].dtype == 'object':
                        from shapely import wkt
                        gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df[geom_col]))
                        
                        geoparquet_file = os.path.join(output_dir, f"{base_filename}.geoparquet")
                        gdf.to_parquet(geoparquet_file)
                        file_size = os.path.getsize(geoparquet_file) / 1024**2
                        print(f"✅ Saved as GeoParquet: {geoparquet_file} ({file_size:.1f} MB)")
                        break
                        
                except Exception as e:
                    print(f"   ❌ Could not process {geom_col} as geometry: {e}")
                    
        except ImportError:
            print(f"   ℹ️  Install geopandas for spatial data support: pip install geopandas")
    
    # 6. Create a summary report
    try:
        summary_file = os.path.join(output_dir, f"{base_filename}_summary.txt")
        with open(summary_file, 'w') as f:
            f.write(f"Delta Sharing Data Export Summary\\n")
            f.write(f"================================\\n\\n")
            f.write(f"Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
            f.write(f"Table: {table_name}\\n")
            f.write(f"Share: {share_name}\\n")
            f.write(f"Schema: {schema_name}\\n\\n")
            f.write(f"Dataset Statistics:\\n")
            f.write(f"- Rows: {len(df):,}\\n")
            f.write(f"- Columns: {len(df.columns)}\\n")
            f.write(f"- Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB\\n")
            f.write(f"- Missing Values: {df.isnull().sum().sum():,}\\n\\n")
            f.write(f"Columns:\\n")
            for i, (col, dtype) in enumerate(df.dtypes.items(), 1):
                f.write(f"{i:2d}. {col} ({dtype})\\n")
            
        print(f"✅ Saved summary report: {summary_file}")
        
    except Exception as e:
        print(f"❌ Summary report save failed: {e}")
    
    print(f"\\n🎉 Data export completed!")
    print(f"📁 All files saved to: {os.path.abspath(output_dir)}")
    
    # List all created files
    created_files = [f for f in os.listdir(output_dir) if f.startswith(base_filename)]
    print(f"\\n📄 Created files:")
    for file in sorted(created_files):
        file_path = os.path.join(output_dir, file)
        file_size = os.path.getsize(file_path) / 1024**2
        print(f"   {file} ({file_size:.1f} MB)")

else:
    print("❌ No data available to save. Please load data first.")

💾 Saving and Exporting Data
📁 Output directory: data/delta-share-downloads

📊 Dataset info:
   Table: d_variabler
   Rows: 2,087,275
   Columns: 17
   Size: 1598.5 MB
   Size: 1598.5 MB

✅ Saved as Parquet: data/delta-share-downloads/d_variabler_20251111_144720.parquet (81.2 MB)

✅ Saved as Parquet: data/delta-share-downloads/d_variabler_20251111_144720.parquet (81.2 MB)
✅ Saved as CSV: data/delta-share-downloads/d_variabler_20251111_144720.csv (263.8 MB)
⚠️  Skipping Excel export (dataset too large: 2,087,275 rows)
✅ Saved as CSV: data/delta-share-downloads/d_variabler_20251111_144720.csv (263.8 MB)
⚠️  Skipping Excel export (dataset too large: 2,087,275 rows)
❌ Metadata save failed: Object of type bool is not JSON serializable
❌ Metadata save failed: Object of type bool is not JSON serializable
✅ Saved summary report: data/delta-share-downloads/d_variabler_20251111_144720_summary.txt
\n🎉 Data export completed!
📁 All files saved to: /home/wilaca/git/miljodir/dp-gaupe-familiegrupper/no

## Summary and Next Steps

This notebook has demonstrated a complete Delta Sharing workflow using native Python.

In [ ]:
# Final summary and recommendations
print("🎯 Delta Sharing Python Integration - Summary")
print("=" * 50)

print(f"\n✅ Successfully demonstrated:")
print(f"   • Native Python Delta Sharing integration")
print(f"   • Automatic handling of advanced Delta features (Deletion Vectors)")
print(f"   • Direct data loading without manual file management")
print(f"   • Comprehensive data exploration and analysis")
print(f"   • Multiple export formats (Parquet, CSV, Excel)")
print(f"   • Security best practices and authentication")

if 'df' in locals() and df is not None:
    print(f"\n📊 Final dataset summary:")
    print(f"   • Table: {table_name}")
    print(f"   • Share: {share_name}")  
    print(f"   • Schema: {schema_name}")
    print(f"   • Rows: {len(df):,}")
    print(f"   • Columns: {len(df.columns)}")
    print(f"   • Data size: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print(f"\n🚀 Advantages of Python approach:")
print(f"   • No R/Python integration complexity")
print(f"   • Full Delta Sharing API access")
print(f"   • Built-in handling of Deletion Vectors")
print(f"   • Rich ecosystem (pandas, matplotlib, seaborn)")
print(f"   • Easy integration with ML pipelines")
print(f"   • Better error handling and debugging")

print(f"\n🛠️  Production recommendations:")
print(f"   1. Use environment variables for credentials")
print(f"   2. Implement automatic token refresh")
print(f"   3. Add retry logic with exponential backoff")
print(f"   4. Monitor token expiration")
print(f"   5. Set up logging for audit trails")
print(f"   6. Consider data caching strategies")
print(f"   7. Implement data validation checks")

print(f"\n📚 Further exploration:")
print(f"   • Explore incremental loading with table versions")
print(f"   • Implement automated data pipelines")
print(f"   • Add integration with data visualization tools")
print(f"   • Set up scheduled data refreshes")
print(f"   • Explore advanced filtering and predicates")

print(f"\n🔧 Required packages for full functionality:")
packages = [
    "delta-sharing>=1.0.0",
    "pandas>=1.3.0", 
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "openpyxl>=3.0.0",  # For Excel export
    "geopandas>=0.10.0"  # For spatial data
]

for package in packages:
    print(f"   • {package}")

print(f"\nInstall command:")
print(f"pip install {' '.join([p.split('>=')[0] for p in packages])}")

print(f"\n✨ Notebook execution completed successfully!")
print(f"📁 Check the 'data/delta-share-downloads' folder for exported files.")